<a href="https://colab.research.google.com/github/kiakass/blog/blob/main/LLaMA3_Fine_tuning_blog2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# A100 환경 및 4비트 양자화를 위한 최신 패키지 설치
!pip install -q -U bitsandbytes transformers accelerate peft datasets trl

############################################################
# 0. Hugging Face 인증
############################################################
from huggingface_hub import notebook_login
notebook_login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 52.2 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# ==========================================
# 1. 데이터셋 및 모델 다운로드 (Setup)
# ==========================================
model_id = "meta-llama/Meta-Llama-3-8B"
# 샘플 데이터셋 (영어 명언 데이터)
dataset = load_dataset("Abirate/english_quotes", split="train[:500]")

# 4비트 양자화 설정 (A100에서 속도와 효율 극대화)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 # A100은 bf16 권장
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [3]:
# 2. Base 모델 로드
print("\n[Step 2] Base 모델 로드 중...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# --- 질문 비교를 위한 테스트 함수 ---
def run_comparison_test(stage_name, questions):
    print(f"\n{'='*30}\n {stage_name} 테스트\n{'='*30}")
    model.eval()
    for i, q in enumerate(questions):
        prompt = f"Quote: {q}\nAuthor:"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=30,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"질문 {i+1}: {q}")
        print(f"답변:\n{response}")
        print("-" * 50)

# 테스트 질문 리스트
test_prompts = [
    "Be yourself; everyone else is already taken.",
    "So many books, so little time.",
    "A room without books is like a body without a soul."
]

# 파인튜닝 전 테스트 실행
run_comparison_test("파인튜닝 전 (Base)", test_prompts)



[Step 2] Base 모델 로드 중...


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]


 파인튜닝 전 (Base) 테스트
질문 1: Be yourself; everyone else is already taken.
답변:
Quote: Be yourself; everyone else is already taken.
Author: Oscar Wilde
Date of Birth: October 16, 1854
Date of Death: November 30, 1900
Country: Ireland
--------------------------------------------------
질문 2: So many books, so little time.
답변:
Quote: So many books, so little time.
Author: The Life of Pi
Awards: Orange Prize
Awards: Pulitzer Prize for Fiction
Awards: Hugo Award
Awards: Pulitzer Prize
--------------------------------------------------
질문 3: A room without books is like a body without a soul.
답변:
Quote: A room without books is like a body without a soul.
Author: Thomas Jefferson
Thomas Jefferson Quotes, Thomas Jefferson Quotes
Quote: A room without books is like a body without a soul.
Author: Thomas Jefferson

--------------------------------------------------


In [4]:
# Finetuning model

from trl import SFTTrainer
from transformers import TrainingArguments # SFTConfig 대신 TrainingArguments 임포트

# 1. 모델 학습 준비 (기존과 동일)
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

# 데이터셋 컬럼 'quote'를 'text'로 변경합니다. (SFTTrainer가 'text'를 기본적으로 찾음)
# 이미 'text' 컬럼이 있는 경우 ValueError를 방지하기 위해 조건부로 실행합니다.
if "quote" in dataset.column_names:
    dataset = dataset.rename_column("quote", "text")

# 2. TrainingArguments 생성 (핵심 수정 부분)
training_args = TrainingArguments(
    output_dir="./llama3_slm_final",
    per_device_train_batch_size=8,   # A100 GPU 권장 배치 사이즈
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,              # 모델 성능 향상을 위해 훈련 에포크 증가
    bf16=True,                       # A100 가속 활성화
    logging_steps=10,
    report_to="none",
    remove_unused_columns=False      # 데이터셋 컬럼 유지를 위해 필수
)

# 3. Trainer 초기화
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=training_args               # 수정된 config 객체 전달
)

print("\n[Step 3] 파인튜닝 시작...")
trainer.train()

# 4. 파인튜닝 후 테스트 실행
run_comparison_test("파인튜닝 후 (Fine-tuned)", test_prompts)

# 5. 파인튜닝된 어댑터 모델 저장
output_dir = "./final_llama3_adapter"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"\n파인튜닝된 모델 어댑터와 토크나이저가 '{output_dir}'에 저장되었습니다.")

Adding EOS to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]


[Step 3] 파인튜닝 시작...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.160200
20,0.909200
30,0.882300
40,0.661300
50,0.628500
60,0.597400
70,0.433200
80,0.422500
90,0.505300



 파인튜닝 후 (Fine-tuned) 테스트
질문 1: Be yourself; everyone else is already taken.
답변:
Quote: Be yourself; everyone else is already taken.
Author: Oscar Wilde
--------------------------------------------------
질문 2: So many books, so little time.
답변:
Quote: So many books, so little time.
Author: J.K. Rowling
--------------------------------------------------
질문 3: A room without books is like a body without a soul.
답변:
Quote: A room without books is like a body without a soul.
Author: Marcus Tullius Cicero
--------------------------------------------------

파인튜닝된 모델 어댑터와 토크나이저가 './final_llama3_adapter'에 저장되었습니다.


한국어 테스트

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# 1. 모델 및 한국어 데이터셋 설정
model_id = "meta-llama/Meta-Llama-3-8B"
# 한국어 위키데이터 QA 데이터셋 (샘플 1000개 추출)
dataset = load_dataset("maywell/ko_wikidata_QA", split="train[:500]")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 # A100 최적화
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 2. 모델 로드 (Base)
print("\n[Step 2] Base 모델 로드 중...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# --- 한국어 질문 비교를 위한 테스트 함수 ---
def run_ko_test(stage_name, questions):
    print(f"\n{'='*30}\n {stage_name} 테스트\n{'='*30}")
    model.eval()
    for i, q in enumerate(questions):
        # 데이터셋의 'instruction'과 'output' 형식을 고려한 프롬프트
        prompt = f"질문: {q}\n답변:"
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                repetition_penalty=1.2 # 한국어 반복 방지
            )

        response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        print(f"테스트 {i+1}. 질문: {q}")
        print(f"결과:\n{response}")
        print("-" * 50)

# 한국어 테스트 질문 3개
ko_questions = [
    "대한민국의 수도는 어디인가요?",
    "세종대왕이 만든 문자의 이름은 무엇인가요?",
    "태양계에서 가장 큰 행성은 무엇인가요?"
]

# 파인튜닝 전 테스트
run_ko_test("파인튜닝 전 (Base)", ko_questions)

# 3. 한국어 데이터 파인튜닝 (LoRA)
print("\n[Step 3] 한국어 데이터 파인튜닝 시작...")
model.train()
model = prepare_model_for_kbit_training(model)

# 한국어 QA 데이터셋의 'instruction'과 'output' 컬럼을 'text' 컬럼으로 변환
def format_korean_qa_dataset(example):
    # 'instruction'과 'output' 컬럼을 결합하여 'text' 컬럼 생성
    example['text'] = f"질문: {example['instruction']}\n답변: {example['output']}"
    return example

# 데이터셋에 'text' 컬럼 생성
dataset = dataset.map(format_korean_qa_dataset)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

# 2. TrainingArguments 생성 (핵심 수정 부분)
training_args = TrainingArguments(
    output_dir="./llama3_korean_slm_final", # 한국어 모델용 출력 디렉토리 변경
    per_device_train_batch_size=8,   # A100 GPU 권장 배치 사이즈
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,              # 모델 성능 향상을 위해 훈련 에포크 증가
    bf16=True,                       # A100 가속 활성화
    logging_steps=10,
    report_to="none",
    remove_unused_columns=False      # 데이터셋 컬럼 유지를 위해 필수
)

# 3. Trainer 초기화
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=training_args               # 수정된 config 객체 전달
)

trainer.train()

# 4. 파인튜닝 후 최종 결과 확인
run_ko_test("파인튜닝 후 (Korean Fine-tuned)", ko_questions)

# 5. 파인튜닝된 어댑터 모델 저장 (한국어 모델용)
output_dir_ko = "./final_llama3_korean_adapter"
trainer.model.save_pretrained(output_dir_ko)
tokenizer.save_pretrained(output_dir_ko)
print(f"\n파인튜닝된 한국어 모델 어댑터와 토크나이저가 '{output_dir_ko}'에 저장되었습니다.")

README.md:   0%|          | 0.00/770 [00:00<?, ?B/s]

data/train.csv:   0%|          | 0.00/144M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/137505 [00:00<?, ? examples/s]


[Step 2] Base 모델 로드 중...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

Adding EOS to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.942300
